In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Philippe,RE,-21.3585,55.7679,72.99,77,39,10.83,light rain
1,1,Hermanus,ZA,-34.4187,19.2345,70.81,49,100,2.33,overcast clouds
2,2,Mackay,AU,-21.1500,149.2000,59.04,56,3,5.30,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,51.01,90,100,8.68,overcast clouds
4,4,Portland,US,45.5234,-122.6762,64.15,83,1,1.01,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
locations = city_data_df[["Lat", "Lng"]]


In [4]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))



In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,64,20,20.71,few clouds
13,13,Hithadhoo,MV,-0.6000,73.0833,81.54,79,55,7.74,broken clouds
14,14,Atuona,PF,-9.8000,-139.0333,78.17,69,1,21.74,clear sky
17,17,Dingle,PH,10.9995,122.6711,77.95,86,12,8.50,light rain
28,28,Novo Aripuana,BR,-5.1206,-60.3797,86.79,45,21,2.46,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                268
City                   268
Country                265
Lat                    268
Lng                    268
Max Temp               268
Humidity               268
Cloudiness             268
Wind Speed             268
Current Description    268
dtype: int64

In [8]:
preferred_cities_df.count()

City_ID                268
City                   268
Country                265
Lat                    268
Lng                    268
Max Temp               268
Humidity               268
Cloudiness             268
Wind Speed             268
Current Description    268
dtype: int64

In [9]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Arraial Do Cabo,BR,-22.9661,-42.0278,75.06,64,20,20.71,few clouds
13,13,Hithadhoo,MV,-0.6000,73.0833,81.54,79,55,7.74,broken clouds
14,14,Atuona,PF,-9.8000,-139.0333,78.17,69,1,21.74,clear sky
17,17,Dingle,PH,10.9995,122.6711,77.95,86,12,8.50,light rain
28,28,Novo Aripuana,BR,-5.1206,-60.3797,86.79,45,21,2.46,few clouds
...,...,...,...,...,...,...,...,...,...,...
665,665,Colares,PT,38.7992,-9.4469,76.19,71,75,10.36,broken clouds
669,669,Orocue,CO,4.7912,-71.3354,76.50,83,100,4.61,overcast clouds
671,671,Morehead,US,37.2711,-87.1764,80.85,87,1,1.01,clear sky
675,675,Loiza,PR,18.4313,-65.8802,85.96,90,40,3.44,scattered clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Arraial Do Cabo,BR,75.06,few clouds,-22.9661,-42.0278,
13,Hithadhoo,MV,81.54,broken clouds,-0.6000,73.0833,
14,Atuona,PF,78.17,clear sky,-9.8000,-139.0333,
17,Dingle,PH,77.95,light rain,10.9995,122.6711,
28,Novo Aripuana,BR,86.79,few clouds,-5.1206,-60.3797,
34,Takoradi,GH,81.01,broken clouds,4.8845,-1.7554,
35,Kuito,AO,75.72,clear sky,-12.3833,16.9333,
40,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,
42,Carutapera,BR,80.98,overcast clouds,-1.1950,-46.0200,
46,Kapaa,US,80.58,moderate rain,22.0752,-159.3190,


In [12]:
len(hotel_df)

268

In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('No hotel found... skipping!')

No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!
No hotel found... skipping!


In [14]:
# 7. Drop the rows where there is no Hotel Name.
#hotel_df[hotel_df['Hotel Name'] == ''] 
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Arraial Do Cabo,BR,75.06,few clouds,-22.9661,-42.0278,Pousada Porto Praia
13,Hithadhoo,MV,81.54,broken clouds,-0.6000,73.0833,Scoop Guest House
14,Atuona,PF,78.17,clear sky,-9.8000,-139.0333,Villa Enata
17,Dingle,PH,77.95,light rain,10.9995,122.6711,SEGAYA PROPERTY
28,Novo Aripuana,BR,86.79,few clouds,-5.1206,-60.3797,Hotel Ezequiel
34,Takoradi,GH,81.01,broken clouds,4.8845,-1.7554,Raybow International Hotel
35,Kuito,AO,75.72,clear sky,-12.3833,16.9333,Herdeiros de Antunes Capusso
40,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,Paradise Inn
42,Carutapera,BR,80.98,overcast clouds,-1.1950,-46.0200,Lidera
46,Kapaa,US,80.58,moderate rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [15]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

In [18]:
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))